# ChatData

## Sqlite
We are using a file system called **sqlite**. It looks and acts like a real single user relational database (RDB). sqlite3 comes packaged with python. You do not need to install the library. You simply import it (as below).

- We are using Pandas to write and read to sqlite. Pandas will manage a lot of the complexity of dealing with a RDB. There are other ways of reading and writing to a RDB that are VERY common and often used in production systems.  

- The most common other way is to read or write to the RDB row by row. As you can see Pandas puts the entire dataframe into the RDB or extracts a new dataframe from the RDB. These are not row by row operations. These are set operations. Set operations ARE more efficient. However, it is common to use row by row operations to avoid needing large memory computers to hold the dataframes. One row only takes a few bytes. An entire dataframe could be many gigabytes and even petabytes of data. Obviously you will end up with problems with your compute resource if your files are this big. This will not happen to you in this activity and may never happen to you while you are a data analyst. Please just be aware of this. 

- This activity is primarily about querying a RDB, so, we are using a simple way that Pandas provides. If you want to learn the more complex way, just google 'reading and wrting to a relational database using python'. There are many resources to learn from.

- The data types that sqlite supports are quite limited. for example, it does not have a DATE type.  This is not a challenge for this project. However, more sophisticated database systems,such as Postgresql, have a large array of data types, such as "DATE", etc. that give those systems additional capability. 

- All of the SQL queries could also be performed on the Pandas DataFrames directly.  You may want to try this yourself for comparison (but make sure you do the SQL queries first, as this is an exercise in using SQL!).


## SQL Magic
Within the Jupyter notebook we will be using something called **SQL Magic**.  This provides a convenient way to write SQL queries directly into code cells in the notebook and to read the results back into a Pandas DataFrame.  This makes working with SQL much easier!

Note that you may need to update your sqlite version using `conda update -c anaconda sqlite` in order for the SQL Magic to work correctly.

## The Data Analysis Lifecycle
The sections in this notebook follow the stages of the Data Analysis Lifecycle introduced in an earlier activity.  The stages are:

- Acquire
- Transform
- Organise
- Analyse
- Communicate
- Maintain


The requirements document for this project is Template SQL queries.xlsx.

# Preliminary Steps: Create a Database
First let's import Sqlite and the other libraries we will need.

In [2]:
# Import Libraries
import numpy as np
import pandas as pd
import sqlite3

### Create the Database
Now we will create the Sqlite database.  Here is some code that does this for you.  We use the `%load_ext` magic command to load the SQL Magic extension and then use `%sql` to connect to the database.

In [3]:
# If the db does not exist, sqlite will create it.
con = sqlite3.connect('chatdata.db')

# loads sql magic
%load_ext sql 

# connects sql magic command to the correct db
%sql sqlite:///chatdata.db 

'Connected: @chatdata.db'

#### Drop the `queries` table if it already exists
The queries table will be our record of the queries created to answer the questions from the requirements spreadsheet.  As we will be running this Jupyter notebook a few times, let's drop (i.e. remove) the queries table so that we start fresh each time.  Here is the code to do this.  We use the `%%sql` magic command to tell Jupyter that we are going to write SQL in the cell.


In [4]:
%%sql
DROP TABLE IF EXISTS 'queries'

 * sqlite:///chatdata.db
Done.


[]

# Task 1: Load the Data
Now we can start loading the data.  The tables will be created as we load the data.  

Note that some of these files are quite large, so make sure you have plenty of free memory!

## Lifecycle Stages: Transform and Organise
The data has already been processed into 3 clean data files ready for this project:

- queries.csv
- posts.csv
- comments.csv

We will load these files into our database.

## Lifecycle Stage: Acquire
The data can be found in the OpenClassrooms instructions for this activity.

### Load Comments Data into a `comments` table
Now we will load the data from the csv files into our sqlite database.

First we load the csv file into a Pandas dataframe:

In [5]:
comments = pd.read_csv('comments.csv')
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


Now take the comments dataframe and push the data into the Sqlite database table called 'comments':

In [6]:
# load comments into sqlite
comments.to_sql('comments', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM comments'
comments = pd.read_sql(sql, con)
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


### Load Other CSVs
Now that you have seen the code for loading in the comments.csv now do the same to read and write the posts.csv and users.csv to sqlite. 



In [7]:
posts = pd.read_csv('posts.csv')
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,NaN,...,NaN,NaN,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,NaN
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,NaN,...,NaN,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,NaN,NaN
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,NaN,...,NaN,NaN,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,NaN,NaN
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,NaN,...,NaN,NaN,2019-08-24 09:57:01,NaN,NaN,0,0,0,NaN,NaN
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,NaN,...,NaN,2019-08-24 12:13:46,2019-08-24 12:13:46,NaN,NaN,0,1,0,NaN,NaN


In [8]:
# load posts into sqlite
posts.to_sql('posts', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM posts'
posts = pd.read_sql(sql, con)
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None


In [9]:
users = pd.read_csv('users.csv')
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,NaN,NaN,NaN,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,NaN,NaN,NaN,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,NaN,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,NaN,NaN,NaN,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,NaN,NaN,NaN,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


In [10]:
# load posts into sqlite
users.to_sql('users', con, if_exists='replace', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM users'
users = pd.read_sql(sql, con)
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


### Drop Duplicates
Look for and drop any duplicates in all 3 of the tables (if they exist). Use Pandas to do this. If you find duplicates, you will need to rewrite the table.

In [11]:
duplicates = comments[comments.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [Id, PostId, Score, Text, CreationDate, UserId]
Index: []


In [12]:
duplicates = posts[posts.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [Id, PostTypeId, AcceptedAnswerId, ParentId, CreationDate, Score, ViewCount, Body, OwnerUserId, OwnerDisplayName, LastEditorUserId, LastEditorDisplayName, LastEditDate, LastActivityDate, Title, Tags, AnswerCount, CommentCount, FavoriteCount, ClosedDate, CommunityOwnedDate]
Index: []

[0 rows x 21 columns]


In [13]:
duplicates = users[users.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [Id, Reputation, CreationDate, DisplayName, LastAccessDate, WebsiteUrl, Location, AboutMe, Views, UpVotes, DownVotes, ProfileImageUrl, AccountId]
Index: []


## Review the Data

Spend some time reviewing the data.  Understand what data we have, think about how that data can be used to assist in the initiative of understanding how ChatData is used in the real world.  Is the data organised in a way that would lend itself to being manged in a relational database?  How would the different tables be connected?  What are the primary and foreign keys?  Would this give you a 3NF model?

Also think about security and ethics.  Is there personal data in here?  Could individuals be identified through this data?  It it ethical to use the data in this way?

In [14]:
users.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'AccountId'],
      dtype='object')

In [15]:
users.head()

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,157607,31,2017-04-17 14:50:42,user157607,2019-07-23 16:44:08,None,None,None,0,0,0,https://www.gravatar.com/avatar/2efb161849efa4...,10705183
1,157656,101,2017-04-17 20:08:20,user102859,2019-06-26 13:42:13,None,None,None,3,0,0,https://i.stack.imgur.com/eY4ka.jpg?s=128&g=1,10567606
2,157704,133,2017-04-18 05:10:47,jupiar,2019-11-25 13:32:27,None,"Shanghai, China","<p>Originally from the U.K, I have an Undergra...",1,1,0,https://www.gravatar.com/avatar/720e20205122c5...,9501631
3,157709,155,2017-04-18 06:39:18,farmer,2019-02-17 19:44:24,None,None,None,16,0,0,https://www.gravatar.com/avatar/0f8c4bde3d8f25...,10709837
4,157755,101,2017-04-18 12:56:17,Miki P,2019-08-12 17:02:21,None,None,None,1,9,0,https://www.gravatar.com/avatar/af088558cd81c5...,7969290


In [16]:
comments.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId'], dtype='object')

In [17]:
comments.head()

,Id,PostId,Score,Text,CreationDate,UserId
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...,2019-01-01 00:06:39,78575
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you...",2019-01-01 00:09:22,2126
2,723186,385137,0,nice. If you felt like doing the work it would...,2019-01-01 00:32:11,2126
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si...",2019-01-01 00:40:36,2126
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^...",2019-01-01 00:41:28,112141


In [18]:
posts.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate'],
      dtype='object')

In [19]:
posts.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,423497,1,423511,0,2019-08-24 09:39:31,2,68,"<p>From wikipedia <a href=""https://en.wikipedi...",64552,None,...,None,None,2019-08-24 11:38:54,When are biased estimators with lower MSE pref...,<bias><unbiased-estimator><mse>,1,0,1,2019-08-25 10:25:24,None
1,423498,1,0,0,2019-08-24 09:47:42,1,24,<p>I am currently doing local sensitivity anal...,87231,None,...,None,2019-09-06 12:52:32,2019-09-06 12:52:32,How to interpret the result from local sensiti...,<sensitivity-analysis><elasticity>,1,0,0,None,None
2,423499,1,0,0,2019-08-24 09:48:26,1,56,<p>I'm an honours student in psychology doing ...,257207,None,...,None,None,2019-08-25 08:54:17,Power Analysis in G-Power - Mixed Model Anova,<anova><gpower>,2,0,0,None,None
3,423500,2,0,215865,2019-08-24 09:57:01,0,0,"<p>Maybe you can look this <a href=""https://me...",106606,None,...,None,None,2019-08-24 09:57:01,None,None,0,0,0,None,None
4,423502,2,0,423286,2019-08-24 10:44:52,3,0,<blockquote>\n <p>Q1) Is my approach valid?</...,220643,None,...,None,2019-08-24 12:13:46,2019-08-24 12:13:46,None,None,0,1,0,None,None


# Working with Sqlite and SQL Magic
In this section let's spend a little time understanding a bit more about how we can work with Sqlite within Jupyter.

Let's look at 2 ways to query the sqlite database: using SQL Magic or using Pandas.  Either way is fine for this project.

## Writing queries with SQL Magic

You will now need to write some queries to get answers to the questions in the requirements.  

For single-line queries, start the cell with `%sql` and simply enter your query:

In [20]:
 %sql

 * sqlite:///chatdata.db


'Connected: @chatdata.db'

For multi line sql statements use `%%sql` as follows. This tells Jupyter that *everything* in this cell should be interpreted as sql. So, NO comments other statements are allowed:

In [21]:
%%sql
SELECT Id, PostId, Score, Text
    FROM comments
        LIMIT 5

 * sqlite:///chatdata.db
Done.


Id,PostId,Score,Text
723182,385124,0,"@BenBolker I don't understand. The fit cannot be done for the negative $y$. So intuitively I'd think that in order to retain the relativity of the data, one'd ideally ""mirror"" or ""flip"" it to the positive axis? Would it be possible to adjust every point individually? Basically e.g. take $abs(y_i)$ instead of minimums."
723183,385124,3,"You can't add *less* than (`-min(y)`), but you could add *more*. I'm going to stop answering now sorry, because **judging what the 'best' approach is depends on much more context that we don't have** (the goal of the analysis, why you need to fit an exponential, your level of computational and statistical sophistication and that of your audience, etc. ...)"
723186,385137,0,"nice. If you felt like doing the work it would be nice to generate an image/contour plot of log-likelihood as a function of (mu, theta) and show the lines corresponding to the two `size` values ."
723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,size=size,log=TRUE)), xlim=c(2,5),ylim=c(0.2,0.5), varnames=c(""mu"",""size""), sys3d=""contour"")abline(h=3.2/7,col=""red"")abline(h=fit.what$theta,col=""blue"")` ... although doesn't look this is actually the answer - mu-hat is independent of theta-hat ... ?"
723188,385134,0,"Don't you mean ""so variance should be $\sigma^2/(n\mu^2)$"""


## Writing queries with Pandas
Another way to write queries is to use pandas:

In [22]:
sql = """
SELECT Id, PostId, Score, Text
    FROM comments
        LIMIT 5
        """
result = pd.read_sql(sql, con)
result

,Id,PostId,Score,Text
0,723182,385124,0,@BenBolker I don't understand. The fit cannot ...
1,723183,385124,3,"You can't add *less* than (`-min(y)`), but you..."
2,723186,385137,0,nice. If you felt like doing the work it would...
3,723187,385137,0,"i.e. `emdbook::curve3d(-sum(dnbinom(y,mu=mu,si..."
4,723188,385134,0,"Don't you mean ""so variance should be $\sigma^..."


# Creating Tables with Referential Integrity

When we loaded the csv files into Sqlite database tables, Sqlite created the tables for us behind the scenes. Let's inspect this a bit more.

We can see how Sqlite created the tables by querying the `sqlite_master` table, which Sqlite uses to keep track of what objects have been created in the database:

In [23]:
%%sql
select sql from sqlite_master

 * sqlite:///chatdata.db
Done.


sql
"CREATE TABLE ""comments"" (""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER)"
"CREATE TABLE ""posts"" (""Id"" INTEGER, ""PostTypeId"" INTEGER, ""AcceptedAnswerId"" INTEGER, ""ParentId"" INTEGER, ""CreationDate"" TEXT, ""Score"" INTEGER, ""ViewCount"" INTEGER, ""Body"" TEXT, ""OwnerUserId"" INTEGER, ""OwnerDisplayName"" TEXT, ""LastEditorUserId"" INTEGER, ""LastEditorDisplayName"" TEXT, ""LastEditDate"" TEXT, ""LastActivityDate"" TEXT, ""Title"" TEXT, ""Tags"" TEXT, ""AnswerCount"" INTEGER, ""CommentCount"" INTEGER, ""FavoriteCount"" INTEGER, ""ClosedDate"" TEXT, ""CommunityOwnedDate"" TEXT)"
"CREATE TABLE ""users"" (""Id"" INTEGER, ""Reputation"" INTEGER, ""CreationDate"" TEXT, ""DisplayName"" TEXT, ""LastAccessDate"" TEXT, ""WebsiteUrl"" TEXT, ""Location"" TEXT, ""AboutMe"" TEXT, ""Views"" INTEGER, ""UpVotes"" INTEGER, ""DownVotes"" INTEGER, ""ProfileImageUrl"" TEXT, ""AccountId"" INTEGER)"


The above results show the `CREATE TABLE` statements that could be used by Sqlite to recreate the tables with the exact same structure.

The problem with the CREATE TABLE statements above is that they don't enforce **referential integrity**.  In other words, they don't ensure that every UserId and PostId in the comments table refers to an actual UserId and PostId in the users and posts tables.  At the moment. we can insert any old number here, and even have multiple users with the same Id!  One of the advantages of working with relational databases is that they can enforce the correct uniqueness and relationships in the data, but at the moment we are not using that feature.  So let's fix that...

First, let's drop the original tables:


In [24]:
%%sql
DROP TABLE comments;
DROP TABLE users;
DROP TABLE posts;

 * sqlite:///chatdata.db
Done.
Done.
Done.


[]

Prove that this worked by selecting the names of the tables back.  We should have no tables:

In [25]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'
    ORDER BY name

 * sqlite:///chatdata.db
Done.


name


In Sqlite we need to enable the enforcement of foreign key constraints:

In [26]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///chatdata.db
Done.


[]

Now recreate the users table with a **primary key constraint** by copying the CREATE TABLE statement from above and adding the NOT NULL PRIMARY KEY clause to the Id:

In [27]:
%%sql
CREATE TABLE "users" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"Reputation" INTEGER,
"CreationDate" TEXT,
"DisplayName" TEXT,
"LastAccessDate" TEXT,
"WebsiteUrl" TEXT,
"Location" TEXT,
"AboutMe" TEXT,
"Views" INTEGER,
"UpVotes" INTEGER,
"DownVotes" INTEGER,
"ProfileImageUrl" TEXT,
"AccountId" INTEGER
);


 * sqlite:///chatdata.db
Done.


[]

Now do the same for the posts table:


In [28]:
%%sql
CREATE TABLE "posts" (
"Id" INTEGER NOT NULL PRIMARY KEY,
"PostTypeId" INTEGER,
"AcceptedAnswerId" INTEGER,
"ParentId" INTEGER,
"CreationDate" TEXT,
"Score" INTEGER,
"ViewCount" INTEGER,
"Body" TEXT,
"OwnerUserId" INTEGER,
"OwnerDisplayName" TEXT,
"LastEditorUserId" INTEGER,
"LastEditorDisplayName" TEXT,
"LastEditDate" TEXT,
"LastActivityDate" TEXT,
"Title" TEXT,
"Tags" TEXT,
"AnswerCount" INTEGER,
"CommentCount" INTEGER,
"FavoriteCount" INTEGER,
"ClosedDate" TEXT,
"CommunityOwnedDate" TEXT
);

 * sqlite:///chatdata.db
Done.


[]

Now for the comments table.  We need to add the primary key constraint on the id here as we did for users and posts, but we also need to add FOREIGN KEY constraints on the UserId and PostId. Read the documentation here and find our how to do that: https://www.sqlite.org/foreignkeys.html.  Then create the comments table with the correct constraints:


In [29]:
%%sql
CREATE TABLE "comments" (
    "Id" INTEGER PRIMARY KEY,
    "PostId" INTEGER,
    "Score" INTEGER,
    "Text" TEXT,
    "CreationDate" TEXT,
    "UserId" INTEGER,
    FOREIGN KEY ("PostId") REFERENCES "posts" ("Id"),
    FOREIGN KEY ("UserId") REFERENCES "users" ("Id")
);

 * sqlite:///chatdata.db
Done.


[]

Now we can re-insert the data into these constrained tables.  First users:

In [30]:
# Insert data into the new users table
users.to_sql('users', con, if_exists='append', index=False)

18412

Now posts:


In [31]:
# Insert data into the new posts table
posts.to_sql('posts', con, if_exists='append', index=False)

42234

Finally comments, which references the users and posts tables:


In [32]:
# Insert data into the new comments table
comments.to_sql('comments', con, if_exists='append', index=False)

50000

Now check that we have the 3 new table definitions in Sqlite:

In [33]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'
    ORDER BY name

 * sqlite:///chatdata.db
Done.


name
comments
posts
users


We now have all the data in tables in Sqlite and the tables will enforce the referential integrity.

Let's see an example of this by answering the following question:

**Which 5 users have viewed the most times and what is the sum of those views per user?**

## Prepare the Sqlite query

First, let's write the query:

In [34]:
sql = """
SELECT Id, SUM(Views) AS TotalViews
    FROM Users
        GROUP BY Id
            ORDER BY TotalViews DESC
                LIMIT 5
                """
result = pd.read_sql(sql, con) # con is the connection to the database
result

,Id,TotalViews
0,919,85180
1,4253,35119
2,805,34637
3,7290,32639
4,3277,29255


## Test the query
You can optionally prove the query worked by performing the same query in Pandas:

In [35]:
results = users.groupby(['Id']).sum().sort_values('Views', ascending = False)[:5]
results['Views']

Id
919     85180
4253    35119
805     34637
7290    32639
3277    29255
Name: Views, dtype: int64

## Insert the query into the `queries` table
Now we need to put this query into the `queries` table in sqlite. Remember we want these queries to be accessible to everybody that should have access to them. We do not want people writing and rewriting the same queires over and over again. The easiest thing to do is create a dictionary with the values and insert these into the queries table.  Note that the values are provided as lists as we are inserting a list of values (i.e. a number of rows) into the table.  In this case the number of rows is 1, so we have lists of 1 item.

So here, we have a column called 'task' with a list of values, a column called 'action' with a list of values, etc.

In [36]:
query_dict = {
              'task': ['Single Table Queries'],
              'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
              'query': [sql]
             }
query_dict

{'task': ['Single Table Queries'],
 'action': ['Which 5 users have viewed the most times and what is the sum of those views per user?'],
 'query': ['\nSELECT Id, SUM(Views) AS TotalViews\n    FROM Users\n        GROUP BY Id\n            ORDER BY TotalViews DESC\n                LIMIT 5\n                ']}

Now that you have the data structure (query_dict) containing the data, create a pandas dataframe that holds those values:

In [37]:
queries = pd.DataFrame(query_dict)
queries

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,"\nSELECT Id, SUM(Views) AS TotalViews\n FRO..."


Now load that pandas dataframe (queries) into the sqlite table called queries. In this case, you use append NOT replace. You will be adding to this tables as you go thru this project.

In [38]:
# load query into sqlite
queries.to_sql('queries', con, if_exists='append', index=False)

# read back in to prove that it worked
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)
queries.head()

,task,action,query
0,Single Table Queries,Which 5 users have viewed the most times and w...,"\nSELECT Id, SUM(Views) AS TotalViews\n FRO..."


It is likely that as you iterate thru this notebook you will create some duplicate entries in the query table. Not a big deal. Just tell sql or pandas (much easier in pandas) to drop the duplicates! If you drop the duplicates in Pandas you will have to write the entire dataframe back to sqlite. Otherwise it is changed in memory in Pandas but NOT in sqlite on the disk drive (or SDD). Just do this drop at the end of this notebook so that you are not constantly dealing with this. 

So, to summarise, as you go through the following tasks you need to:

- answer the question in sql
- prove it in pandas (if you want to)
- put the query into the queries table

##  Insert the CREATE TABLE Statements into the `queries` Table

Now that we understand how to populate the `queries` table, let's insert the CREATE TABLE statements into it.  First let's define a function to help us insert into the queries table:


In [39]:
def store_query(task, action, query):
    query_dict = {
                  'task': [task],
                  'action': [action],
                  'query': [query]
                 }

    # put query into the query_dict
    queries = pd.DataFrame(query_dict)

    # load query into sqlite
    queries.to_sql('queries', con, if_exists='append', index=False)    

Now we can specify the queries and call the above function to store them.  The first one is done for you.

In [40]:
sql = """
    CREATE TABLE "comments" (
    "Id" INTEGER,
    "PostId" INTEGER,
    "Score" INTEGER,
    "Text" TEXT,
    "CreationDate" TEXT,
    "UserId" INTEGER
    )
    """
store_query("Task 1", "Create table comments", sql)

Let's prove it works by selecting back from the queries table:

In [41]:
# Prove it works
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"


## Insert the other CREATE TABLE statements into the `queries` table.
Follow the above pattern to complete these code cells:


In [42]:
# Insert the CREATE TABLE for posts into the queries table
sql = """
    CREATE TABLE "posts" (
"Id" INTEGER,
"PostTypeId" INTEGER,
"AcceptedAnswerId" INTEGER,
"ParentId" INTEGER,
"CreationDate" TEXT,
"Score" INTEGER,
"ViewCount" INTEGER,
"Body" TEXT,
"OwnerUserId" INTEGER,
"OwnerDisplayName" TEXT,
"LastEditorUserId" INTEGER,
"LastEditorDisplayName" TEXT,
"LastEditDate" TEXT,
"LastActivityDate" TEXT,
"Title" TEXT,
"Tags" TEXT,
"AnswerCount" INTEGER,
"CommentCount" INTEGER,
"FavoriteCount" INTEGER,
"ClosedDate" TEXT,
"CommunityOwnedDate" TEXT
)
    """
store_query("Task 2", "Create table posts", sql)

In [43]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table posts,"CREATE TABLE ""posts"" (""Id"" INTEGER,""PostTypeId"" INTEGER,""AcceptedAnswerId"" INTEGER,""ParentId"" INTEGER,""CreationDate"" TEXT,""Score"" INTEGER,""ViewCount"" INTEGER,""Body"" TEXT,""OwnerUserId"" INTEGER,""OwnerDisplayName"" TEXT,""LastEditorUserId"" INTEGER,""LastEditorDisplayName"" TEXT,""LastEditDate"" TEXT,""LastActivityDate"" TEXT,""Title"" TEXT,""Tags"" TEXT,""AnswerCount"" INTEGER,""CommentCount"" INTEGER,""FavoriteCount"" INTEGER,""ClosedDate"" TEXT,""CommunityOwnedDate"" TEXT)"


In [44]:
# Insert the CREATE TABLE for users into the queries table
sql = """
  CREATE TABLE "users" (
"Id" INTEGER,
"Reputation" INTEGER,
"CreationDate" TEXT,
"DisplayName" TEXT,
"LastAccessDate" TEXT,
"WebsiteUrl" TEXT,
"Location" TEXT,
"AboutMe" TEXT,
"Views" INTEGER,
"UpVotes" INTEGER,
"DownVotes" INTEGER,
"ProfileImageUrl" TEXT,
"AccountId" INTEGER
)
    """
store_query("Task 3", "Create table users", sql)

In [45]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table posts,"CREATE TABLE ""posts"" (""Id"" INTEGER,""PostTypeId"" INTEGER,""AcceptedAnswerId"" INTEGER,""ParentId"" INTEGER,""CreationDate"" TEXT,""Score"" INTEGER,""ViewCount"" INTEGER,""Body"" TEXT,""OwnerUserId"" INTEGER,""OwnerDisplayName"" TEXT,""LastEditorUserId"" INTEGER,""LastEditorDisplayName"" TEXT,""LastEditDate"" TEXT,""LastActivityDate"" TEXT,""Title"" TEXT,""Tags"" TEXT,""AnswerCount"" INTEGER,""CommentCount"" INTEGER,""FavoriteCount"" INTEGER,""ClosedDate"" TEXT,""CommunityOwnedDate"" TEXT)"
Task 3,Create table users,"CREATE TABLE ""users"" (""Id"" INTEGER,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER)"


## Count the Number of Rows in Each Table
Run some queries to count the number of rows in each of the tables.  Don't forget to insert the query into the queries table.


In [46]:
# Count the number of rows in the comments table
%sql SELECT COUNT(*)  FROM comments;

 * sqlite:///chatdata.db
Done.


COUNT(*)
50000


In [47]:
# Count the number of rows in the users table
%sql SELECT COUNT(*)  FROM users;

 * sqlite:///chatdata.db
Done.


COUNT(*)
18412


In [48]:
# Count the number of rows in the posts table
%sql SELECT COUNT(*)  FROM posts;

 * sqlite:///chatdata.db
Done.


COUNT(*)
42234


##  Random Checks on the Data


In [49]:
# Run the query to select 5 random rows from the posts table
sql = """
select * from Comments order by random() limit 5;
"""
store_query("Task 4", "Random Comments", sql)
result = pd.read_sql(sql, con)
result

,Id,PostId,Score,Text,CreationDate,UserId
0,740140,393255,0,"Thanks again. In the meantime, I found a packa...",2019-02-25 22:14:29,174439
1,779232,417559,0,"Right, it's so each subject has their own smal...",2019-07-15 19:23:13,253228
2,723966,385548,0,"Thanks. 1- In this case, should I test both pa...",2019-01-04 04:14:40,232988
3,772904,414055,0,I based it off this previous question: https:/...,2019-06-21 04:45:54,236180
4,748317,399546,0,There are many conflicting ways of choosing st...,2019-03-26 18:47:21,53690


In [50]:
# Run the query to select 5 random rows from the posts comments
sql = """
select * from Users order by random() limit 5;
"""
store_query("Task 5", "Random post Users", sql)
result = pd.read_sql(sql, con)
result

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,ProfileImageUrl,AccountId
0,3601,6428,2011-03-07 21:51:11,Aaron - Reinstate Monica,2019-11-19 18:30:52,http://www.stat.umn.edu/~arendahl/,"St. Paul, MN",<p>I am an Assistant Professor of Statistics a...,1135,459,3,https://i.stack.imgur.com/fGuXb.png?s=128&g=1,73081
1,244476,125,2019-04-12 00:36:56,jytoronto,2019-07-25 00:36:37,None,None,None,1,3,0,https://www.gravatar.com/avatar/a09ce10c684d43...,2651087
2,246828,11,2019-05-03 22:00:55,viggen,2019-10-16 21:33:42,None,None,None,1,0,0,https://www.gravatar.com/avatar/772e22c3da65cf...,15868583
3,203672,115,2018-04-09 20:07:18,Michael Stachowsky,2019-05-15 10:54:45,None,Canada,<p>Engineering instructor and part-time mathem...,2,1,0,https://lh5.googleusercontent.com/-a-UIp5V6iDM...,6379999
4,234145,11,2019-01-15 15:33:40,adeevee,2019-09-23 17:14:00,None,"New York, NY, USA",None,2,0,0,https://www.gravatar.com/avatar/966a0a7c414535...,15129326


In [51]:
# Run the query to select 5 random rows from the users table
sql = """
select * from Posts order by random() limit 5;
"""
store_query("Task 6", "Random post Posts", sql)
result = pd.read_sql(sql, con)
result

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,394325,2,0,394312,2019-02-25 18:26:22,1,0,"<p>I see some normality in this dataset, but I...",171498,None,...,None,None,2019-02-25 18:26:22,None,None,0,3,0,None,None
1,435000,1,0,0,2019-11-07 12:04:08,4,50,"<p>I have a regression task, for which I'm tra...",203565,None,...,None,None,2019-11-07 13:58:16,Loss function for regression with uncertain la...,<regression><machine-learning><loss-functions>,1,0,1,None,None
2,412299,2,0,412276,2019-06-10 12:47:54,5,0,<p>The problem with the mass-fitting packages ...,3382,None,...,None,2019-06-10 13:23:57,2019-06-10 13:23:57,None,None,0,3,0,None,None
3,406973,1,406975,0,2019-05-07 06:09:40,4,965,<p>I am doing time-series analysis in python f...,208706,None,...,None,2019-05-07 07:01:08,2019-05-07 12:43:29,How to choose between ARIMA and ARMA model,<time-series><python><arima><stationarity>,2,1,3,None,None
4,407915,2,0,407911,2019-05-12 04:40:36,17,0,"<p>It is per 100,000 people in the population ...",56133,None,...,None,2019-05-12 06:43:54,2019-05-12 06:43:54,None,None,0,3,0,None,None


# Create Single Table Queries


## Lifecycle Stage: Analyze
We can now start the analysis with our single-table queries.  First we need to create a new computed column to help with one of the queries.  The code below creates a column called LEN_BODY which is the length of the BODY text:


In [52]:
%%sql
ALTER TABLE POSTS ADD COLUMN LEN_BODY INT

 * sqlite:///chatdata.db
Done.


[]

In [53]:
%%sql
UPDATE POSTS SET LEN_BODY = LENGTH(BODY)

 * sqlite:///chatdata.db
42234 rows affected.


[]

### Single Table Queries
From the Template SQL Queries there are a series of queries requested. You are responsible for coding the result. Use the pattern above to accomplish the coding objective.  Copy and paste the question (column B Action) into a markdown cell and answer the query. Use the pattern that you have been using above. 



How many posts have 0 comments?

In [54]:
sql = """
SELECT COUNT(*) FROM Posts WHERE CommentCount = 0;
"""
store_query("Task 7", "How many posts have 0 comments", sql)
result = pd.read_sql(sql, con)
result

,COUNT(*)
0,21713


In [55]:
sql = """
SELECT COUNT(*) FROM Posts WHERE CommentCount = 1;
"""
store_query("Task 8", "How many posts have 1 comments", sql)
result = pd.read_sql(sql, con)
result

,COUNT(*)
0,6460


In [56]:
sql = """
SELECT COUNT(*) FROM Posts WHERE CommentCount >= 2;
"""
store_query("Task 7", "How many posts have 0 comments", sql)
result = pd.read_sql(sql, con)
result

,COUNT(*)
0,14061


In [57]:
sql = """
SELECT Id, Title, ViewCount
FROM Posts
ORDER BY ViewCount DESC
LIMIT 5;
"""

store_query("Task 8", "Find the 5 posts with the highest ViewCount", sql)
result = pd.read_sql(sql, con)
result

,Id,Title,ViewCount
0,388566,"Is it wrong to rephrase ""1 in 80 deaths is cau...",19542
1,394118,Why do neural networks need so many training e...,16317
2,431370,Is there a name for the phenomenon of false po...,11723
3,398646,"What does ""Scientists rise up against statisti...",9850
4,434128,Famous statistical wins and horror stories for...,6718


In [58]:
sql = """
SELECT Id, Title, Score
FROM Posts
ORDER BY Score DESC
LIMIT 5;
"""
store_query("Task 9", "Find the 5 posts with the highest Score", sql)
result = pd.read_sql(sql, con)
result

,Id,Title,Score
0,431397,None,101
1,394128,None,100
2,426878,None,93
3,388578,None,80
4,431370,Is there a name for the phenomenon of false po...,77


In [59]:
sql = """
SELECT Score, COUNT(*) AS count
FROM Posts
GROUP BY Score
ORDER BY count DESC
LIMIT 5;
"""
store_query("Task 10", "5 most frequent Scores", sql)
result = pd.read_sql(sql, con)
result

,Score,count
0,0,19888
1,1,11867
2,2,5094
3,3,2228
4,4,1059


In [60]:
sql = """
SELECT COUNT(*) 
FROM Posts 
WHERE Tags LIKE '%data%';
"""
store_query("Task 11", "How Many keyword Data in Tags", sql)
result = pd.read_sql(sql, con)
result

,COUNT(*)
0,2242


In [61]:
sql = """
SELECT CommentCount, COUNT(*) AS Frequency
FROM Posts
GROUP BY CommentCount
ORDER BY Frequency DESC
LIMIT 5;
"""
store_query("Task 12", "Most frequent commentcount for posts", sql)
result = pd.read_sql(sql, con)
result

,CommentCount,Frequency
0,0,21713
1,1,6460
2,2,4966
3,3,3063
4,4,2026


In [62]:
sql = """
SELECT COUNT(*) as AcceptedAnswerCount 
FROM Posts 
WHERE AcceptedAnswerId >0;
"""
store_query("Task 13", "How many posts have an accepted answer?", sql)
result = pd.read_sql(sql, con)
result

,AcceptedAnswerCount
0,5341


In [63]:
sql = """
SELECT AVG(reputation) AS avg_reputation FROM users;
"""
store_query("Task 14", "Average reputation of users", sql)
result = pd.read_sql(sql, con)
result

,avg_reputation
0,312.350912


In [64]:
sql = """
SELECT MIN(reputation), MAX(reputation) FROM users;
"""
store_query("Task 14", "Min and Max reputation of users", sql)
result = pd.read_sql(sql, con)
result

,MIN(reputation),MAX(reputation)
0,1,228662


In [65]:
sql = """
SELECT LEN_BODY, ViewCount
FROM Posts
ORDER BY ViewCount DESC
LIMIT 5
"""
store_query("Task 15", "What is the length of the body of 5 most viewed posts?", sql)
result = pd.read_sql(sql, con)
result

,LEN_BODY,ViewCount
0,2270,19542
1,512,16317
2,811,11723
3,2148,9850
4,1172,6718


In [66]:
sql = """
SELECT COUNT(DISTINCT (location)) AS num_locations
FROM users;
"""
store_query("Task 16", "Distinct Locations", sql)
result = pd.read_sql(sql, con)
result

,num_locations
0,1900


In [67]:
sql = """
SELECT Location, COUNT(*) AS count
FROM Users
WHERE Location IS NOT NULL
GROUP BY Location
ORDER BY count DESC
LIMIT 5
"""
store_query("Task 17", "top 5 locations for users", sql)
result = pd.read_sql(sql, con)
result

,Location,count
0,Germany,117
1,India,100
2,United States,69
3,"Paris, France",66
4,"London, United Kingdom",63


In [68]:
sql = """
SELECT CAST(strftime('%w', CreationDate) AS INTEGER) AS day_of_week,
       SUM(ViewCount) AS total_views,
       100.0 * SUM(ViewCount) / (SELECT SUM(ViewCount) FROM Posts) AS percentage
FROM Posts
GROUP BY day_of_week
ORDER BY total_views DESC
"""
store_query("Task 18", "Rank Days of the week", sql)
result = pd.read_sql(sql, con)
result

,day_of_week,total_views,percentage
0,4,331507,16.819476
1,3,330506,16.768689
2,2,320382,16.255034
3,1,311546,15.806727
4,5,267324,13.563061
5,0,234459,11.895609
6,6,175247,8.891404


#  Cross Table Queries

## Lifecycle Stage: Analyze
Let's continue the analysis with our multi-table queries.  

### Cross Table Queries

From the Template SQL Queries there are a series of queries requested. You are responsible for coding the result. Use the pattern above to accomplish the coding objective.  Copy and paste the question (column B Action) into a markdown cell and answer the query. Use the pattern that you have been using above. 


In [69]:
sql = """
SELECT COUNT(*) AS post_count
FROM users
JOIN posts ON users.Id = posts.OwnerUserId
WHERE users.AboutMe IS NOT NULL;
"""
store_query("Task 3.1", "Post created by a user about me section", sql)
result = pd.read_sql(sql, con)
result

,post_count
0,17189


In [70]:
sql = """
SELECT COUNT(*) / (SELECT CAST(COUNT(*) AS FLOAT) FROM USERS WHERE ABOUTME !='') AS posts_per_user
    FROM POSTS P
    JOIN USERS U ON P.OWNERUSERID = U.ID
    WHERE U.ABOUTME !='';
    """
store_query("Task 3", "Considering only the users with an 'AboutMe', how many posts are there per user?", sql)
result = pd.read_sql(sql, con)
result.head()

,posts_per_user
0,4.274807


In [321]:
sql = """ 
SELECT posts.Id, posts.Title, COUNT(comments.Id) AS comment_count
FROM posts
JOIN comments ON posts.Id = comments.PostId
GROUP BY posts.Id
ORDER BY comment_count DESC
LIMIT 10;
"""
store_query("Task 3.3", "Top 10 posts number of comments", sql)
result = pd.read_sql(sql, con)
result

,Id,Title,comment_count
0,386853,None,66
1,386556,Strategies for predicting 100 binary choices g...,34
2,418910,None,31
3,395232,None,31
4,402987,Evaluating Unbalanced Multiclass Classifiers: ...,27
5,386075,Avoiding social discrimination in model building,26
6,394118,Why do neural networks need so many training e...,24
7,402950,Poisson Regression in R with individual fixed-...,23
8,398828,None,23
9,396111,How do I intepret these t-SNE results?,22


In [322]:
sql = """ 
SELECT posts.Id, posts.Title, posts.Score + SUM(comments.Score) AS total_score
FROM posts
JOIN comments ON posts.Id = comments.PostId
GROUP BY posts.Id
ORDER BY total_score DESC
LIMIT 10;

"""
store_query("Task 3.4", "Top 10 Score", sql)
result = pd.read_sql(sql, con)
result

,Id,Title,total_score
0,394118,Why do neural networks need so many training e...,306
1,394128,None,169
2,388578,None,141
3,398653,None,111
4,388566,"Is it wrong to rephrase ""1 in 80 deaths is cau...",101
5,398646,"What does ""Scientists rise up against statisti...",99
6,421677,None,96
7,420526,None,92
8,400317,None,72
9,388582,None,69


In [323]:
sql = """ 
SELECT u.Id AS user_id, COUNT(c.Id) AS comment_count, u.Reputation AS user_reputation
FROM users u
JOIN comments c ON u.Id = c.UserId
GROUP BY u.Id
ORDER BY user_reputation DESC
LIMIT 10;
"""
store_query("Task 3.5.1", "Top 10 Most comments", sql)
result = pd.read_sql(sql, con)
result

,user_id,comment_count,user_reputation
0,805,1153,228662
1,919,3301,223056
2,7290,370,115531
3,686,349,85077
4,28666,158,75024
5,35989,470,71548
6,7224,424,65999
7,4253,255,59952
8,1352,422,59160
9,22311,467,51155


In [324]:
sql = """ 
SELECT u.Id AS user_id, COUNT(c.Id) AS comment_count, u.Reputation AS user_reputation
FROM users u
JOIN comments c ON u.Id = c.UserId
GROUP BY u.Id
ORDER BY comment_count DESC
LIMIT 10;
"""
store_query("Task 3.5.2", "Top 10 Most comments", sql)
result = pd.read_sql(sql, con)
result

,user_id,comment_count,user_reputation
0,919,3301,223056
1,805,1153,228662
2,143489,1024,2890
3,11887,805,39200
4,85665,691,17391
5,164061,540,13485
6,22047,536,41385
7,158565,504,6482
8,7962,492,8030
9,35989,470,71548


In [325]:
#COMMENT -

In [326]:
sql = """ 
SELECT u.Id AS user_id, COUNT(p.Id) AS post_count, u.Reputation AS user_reputation
FROM users u
JOIN posts p ON u.Id = p.OwnerUserId
GROUP BY u.Id
ORDER BY post_count DESC
LIMIT 10;
"""
store_query("Task 3.6.1", "Top 10 users that post the most", sql)
result = pd.read_sql(sql, con)
result

,user_id,post_count,user_reputation
0,204068,637,17404
1,85665,545,17391
2,173082,435,42553
3,11887,435,39200
4,686,386,85077
5,1352,285,59160
6,3382,274,24841
7,7224,233,65999
8,35989,230,71548
9,805,230,228662


In [327]:
sql = """ 
SELECT u.Id AS user_id, COUNT(p.Id) AS post_count, u.Reputation AS user_reputation
FROM users u
JOIN posts p ON u.Id = p.OwnerUserId
GROUP BY u.Id
ORDER BY user_reputation DESC
LIMIT 10;
"""
store_query("Task 3.6.2", "Top 10 users that post the most", sql)
result = pd.read_sql(sql, con)
result

,user_id,post_count,user_reputation
0,805,230,228662
1,919,203,223056
2,7290,35,115531
3,686,386,85077
4,28666,8,75024
5,35989,230,71548
6,7224,233,65999
7,4253,71,59952
8,1352,285,59160
9,22311,140,51155


In [328]:
#COMMENT

#  Check the Queries Table

Now let's tidy up and check the queries table.

First let's check it's contents:

In [329]:
%sql SELECT * FROM queries

 * sqlite:///chatdata.db
Done.


task,action,query
Single Table Queries,Which 5 users have viewed the most times and what is the sum of those views per user?,"SELECT Id, SUM(Views) AS TotalViews FROM Users GROUP BY Id ORDER BY TotalViews DESC LIMIT 5"
Task 1,Create table comments,"CREATE TABLE ""comments"" ( ""Id"" INTEGER, ""PostId"" INTEGER, ""Score"" INTEGER, ""Text"" TEXT, ""CreationDate"" TEXT, ""UserId"" INTEGER )"
Task 2,Create table posts,"CREATE TABLE ""posts"" (""Id"" INTEGER,""PostTypeId"" INTEGER,""AcceptedAnswerId"" INTEGER,""ParentId"" INTEGER,""CreationDate"" TEXT,""Score"" INTEGER,""ViewCount"" INTEGER,""Body"" TEXT,""OwnerUserId"" INTEGER,""OwnerDisplayName"" TEXT,""LastEditorUserId"" INTEGER,""LastEditorDisplayName"" TEXT,""LastEditDate"" TEXT,""LastActivityDate"" TEXT,""Title"" TEXT,""Tags"" TEXT,""AnswerCount"" INTEGER,""CommentCount"" INTEGER,""FavoriteCount"" INTEGER,""ClosedDate"" TEXT,""CommunityOwnedDate"" TEXT)"
Task 3,Create table users,"CREATE TABLE ""users"" (""Id"" INTEGER,""Reputation"" INTEGER,""CreationDate"" TEXT,""DisplayName"" TEXT,""LastAccessDate"" TEXT,""WebsiteUrl"" TEXT,""Location"" TEXT,""AboutMe"" TEXT,""Views"" INTEGER,""UpVotes"" INTEGER,""DownVotes"" INTEGER,""ProfileImageUrl"" TEXT,""AccountId"" INTEGER)"
Task 4,Random Comments,select * from Comments order by random() limit 5;
Task 5,Random post Users,select * from Users order by random() limit 5;
Task 6,Random post Posts,select * from Posts order by random() limit 5;
Task 7,How many posts have 0 comments,SELECT COUNT(*) FROM Posts WHERE CommentCount = 0;
Task 8,How many posts have 1 comments,SELECT COUNT(*) FROM Posts WHERE CommentCount = 1;
Task 7,How many posts have 0 comments,SELECT COUNT(*) FROM Posts WHERE CommentCount >= 2;


## Drop Duplicates

You likely have some duplicates. Lets drop them. 

In [330]:
# Read the queries table into pandas
sql = 'SELECT * FROM queries'
queries = pd.read_sql(sql, con)

# Drop duplicates
queries.drop_duplicates(inplace = True) # drop duplicates

## Case Issues

Remember, SQL is case insensitive. Pandas IS case sensitive. Lets deal with this now by making all of the text uppercase. 

In [331]:
for col in queries.columns:
    queries[col] = queries[col].str.upper()
    
queries

,task,action,query
0,SINGLE TABLE QUERIES,WHICH 5 USERS HAVE VIEWED THE MOST TIMES AND W...,"\nSELECT ID, SUM(VIEWS) AS TOTALVIEWS\n FRO..."
1,TASK 1,CREATE TABLE COMMENTS,"\n CREATE TABLE ""COMMENTS"" (\n ""ID"" INTE..."
2,TASK 2,CREATE TABLE POSTS,"\n CREATE TABLE ""POSTS"" (\n""ID"" INTEGER,\n""..."
3,TASK 3,CREATE TABLE USERS,"\n CREATE TABLE ""USERS"" (\n""ID"" INTEGER,\n""RE..."
4,TASK 4,RANDOM COMMENTS,\nSELECT * FROM COMMENTS ORDER BY RANDOM() LIM...
5,TASK 5,RANDOM POST USERS,\nSELECT * FROM USERS ORDER BY RANDOM() LIMIT ...
6,TASK 6,RANDOM POST POSTS,\nSELECT * FROM POSTS ORDER BY RANDOM() LIMIT ...
7,TASK 7,HOW MANY POSTS HAVE 0 COMMENTS,\nSELECT COUNT(*) FROM POSTS WHERE COMMENTCOUN...
8,TASK 8,HOW MANY POSTS HAVE 1 COMMENTS,\nSELECT COUNT(*) FROM POSTS WHERE COMMENTCOUN...
9,TASK 7,HOW MANY POSTS HAVE 0 COMMENTS,\nSELECT COUNT(*) FROM POSTS WHERE COMMENTCOUN...


In [332]:
# Write the now deduped uppercase dataframe back to sqlite and replace the table
queries.to_sql('queries', con, if_exists='replace', index=False)

30

## Use Case

Now that we have this queries table, lets give you some ideas about how you would use it.

Suppose you wanted to find all of the queries where you did a GROUP BY:

In [333]:
%%sql
SELECT query
    FROM queries
        WHERE query LIKE '%GROUP BY%'

 * sqlite:///chatdata.db
Done.


query
"SELECT ID, SUM(VIEWS) AS TOTALVIEWS FROM USERS GROUP BY ID ORDER BY TOTALVIEWS DESC LIMIT 5"
"SELECT SCORE, COUNT(*) AS COUNTFROM POSTSGROUP BY SCOREORDER BY COUNT DESCLIMIT 5;"
"SELECT COMMENTCOUNT, COUNT(*) AS FREQUENCYFROM POSTSGROUP BY COMMENTCOUNTORDER BY FREQUENCY DESCLIMIT 5;"
"SELECT LOCATION, COUNT(*) AS COUNTFROM USERSWHERE LOCATION IS NOT NULLGROUP BY LOCATIONORDER BY COUNT DESCLIMIT 5"
"SELECT CAST(STRFTIME('%W', CREATIONDATE) AS INTEGER) AS DAY_OF_WEEK, SUM(VIEWCOUNT) AS TOTAL_VIEWS, 100.0 * SUM(VIEWCOUNT) / (SELECT SUM(VIEWCOUNT) FROM POSTS) AS PERCENTAGEFROM POSTSGROUP BY DAY_OF_WEEKORDER BY TOTAL_VIEWS DESC"
"SELECT USERS.ID, USERS.DISPLAYNAME, COUNT(POSTS.ID) AS POST_COUNTFROM USERSJOIN POSTS ON USERS.ID = POSTS.OWNERUSERIDWHERE USERS.ABOUTME IS NOT NULLGROUP BY USERS.IDORDER BY POST_COUNT DESC;"
"SELECT POSTS.ID, POSTS.TITLE, COUNT(COMMENTS.ID) AS COMMENT_COUNTFROM POSTSJOIN COMMENTS ON POSTS.ID = COMMENTS.POSTIDGROUP BY POSTS.IDORDER BY COMMENT_COUNT DESCLIMIT 10;"
"SELECT POSTS.ID, POSTS.TITLE, POSTS.SCORE + SUM(COMMENTS.SCORE) AS TOTAL_SCOREFROM POSTSJOIN COMMENTS ON POSTS.ID = COMMENTS.POSTIDGROUP BY POSTS.IDORDER BY TOTAL_SCORE DESCLIMIT 10;"
"SELECT U.ID AS USER_ID, COUNT(C.ID) AS COMMENT_COUNT, U.REPUTATION AS USER_REPUTATIONFROM USERS UJOIN COMMENTS C ON U.ID = C.USERIDGROUP BY U.IDORDER BY USER_REPUTATION DESCLIMIT 10;"
"SELECT U.ID AS USER_ID, COUNT(C.ID) AS COMMENT_COUNT, U.REPUTATION AS USER_REPUTATIONFROM USERS UJOIN COMMENTS C ON U.ID = C.USERIDGROUP BY U.IDORDER BY COMMENT_COUNT DESCLIMIT 10;"



Find the queries that have 'DISTINCT' in them. You can do it with the %sql command or with Pandas and sql.


In [335]:
%%sql
SELECT query
    FROM queries
        WHERE query LIKE '%DISTINCT%'

 * sqlite:///chatdata.db
Done.


query
SELECT COUNT(DISTINCT (LOCATION)) AS NUM_LOCATIONSFROM USERS;


In [336]:
con.close()